# Домашнее задание 3#

In [46]:
import numpy as np
import matplotlib.pyplot as plt

In [63]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [64]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [65]:
#Стандартизируем третий признак
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [66]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [91]:
#def calc_logloss(y, y_pred):
#    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
#    return err

In [101]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
        print(i, W, err)
    return W

In [102]:
W = eval_model(X_st, y, iterations=1000, alpha=1e-5)

1000 [ 0.49282757 -0.15007512  0.64748969  1.51727928] 1.2013133824878832


**1.Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

In [96]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred-1e-10))
    return err

**2.Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

In [145]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        y_pred_proba = calc_pred_proba(W, X)
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return y_pred_proba

In [131]:
eval_model(X_st, y, iterations=90000, alpha=1e-1)

9000 [-10.67239108  -1.40571631  -2.1950503    9.01453862] 0.2586628760913193
18000 [-15.35833092  -1.7937128   -3.63759762  12.79206658] 0.21561207626459886
27000 [-19.0399625   -2.11133453  -4.74241755  15.77840201] 0.18908459199222447
36000 [-22.1114095   -2.37992046  -5.65152931  18.27023302] 0.1706627560670153
45000 [-24.74975994  -2.61214065  -6.42523187  20.4090502 ] 0.1570977823833049
54000 [-27.06439659  -2.81649645  -7.09951522  22.2834429 ] 0.1466764127491592
63000 [-29.12894501  -2.99893696  -7.69804565  23.95328422] 0.13839908739524132
72000 [-30.99523271  -3.16375265  -8.23719474  25.46079233] 0.1316455437068398
81000 [-32.70096576  -3.31411958  -8.72871041  26.83671018] 0.12601215907732385
90000 [-34.27432938  -3.45244383  -9.1812758   28.10403169] 0.1212257983013518


**3.Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**

In [155]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(W,X))
    return y_pred_proba

In [150]:
calc_pred_proba(W, X_st)

array([0.28964718, 0.13239018, 0.99730794, 0.19964039, 0.73521642,
       0.28535565, 0.99936586, 0.08615267, 0.34267961, 0.99086206])

**4.Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

In [170]:
def calc_pred(W, X, t):
    y_pred = calc_pred_proba(W, X)
    y_pred = np.where(y_pred > t, 1, 0)
    return y_pred

In [198]:
#Порог 0.3 предсказывает со 100% точностью, но для интереса берем 0.5
y_pred = calc_pred(W, X_st, 0.5)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

In [199]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

**5.Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.**

In [265]:
def score(y, y_pred):
    
    accuracy = sum(y_pred==y)/len(y)
    
    conf_matrix = np.zeros(shape=(2,2))
    tp = np.sum(np.where(y==1,y==y_pred,False))
    fp = np.sum(np.where(y==0,y==y_pred,False))
    fn = np.sum(np.where(y==1,y!=y_pred,False))
    tn = np.sum(np.where(y==0,y!=y_pred,False))


    conf_matrix[0,0] = tp
    conf_matrix[1,1] = fp
    conf_matrix[1,0] = fn
    conf_matrix[0,1] = tn

    preсision = tp /(tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * preсision * recall / (preсision + recall)
    
    
    print(f"conf_matrix = \n {conf_matrix.astype(int)} , \n accuracy = {accuracy}, \n preсision = {preсision}, \n recall = {recall}, \n f1 = {f1}", sep='\n')


In [266]:
score(y, y_pred)

conf_matrix = 
 [[4 0]
 [1 5]] , 
 accuracy = 0.9, 
 preсision = 0.4444444444444444, 
 recall = 0.8, 
 f1 = 0.5714285714285714


**6.Могла ли модель переобучиться? Почему?**

Модель вероятнее всего переобучиась. Для проверки этой гипотезы необходимо провести предсказания на тестовой выборке.
На переобучение указывают очень большие веса.
Причиной этому могли стать выбросы во втором признаке. В нашем случае необходимо применить регуляризацию, т.к. избавляться от выбросов при столь малой выборке кощунство. 

**7. (опция) Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.**

In [299]:
def eval_model_l1(X, y, iterations, alpha, lambda_):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [308]:
#При прочих равных сработала лучше, чем L2
eval_model_l1(X_st, y, iterations=50000, alpha=1e-1, lambda_ = 1e-2)

5000 [-4.65803321 -1.01279537 -0.15793214  4.442434  ] 0.3523854270112542
10000 [-5.17726051 -1.03315282 -0.34593351  4.80093517] 0.3405157199050367
15000 [-5.31242493 -1.03901688 -0.39414763  4.89575086] 0.33760709000315775
20000 [-5.35001011 -1.04068757 -0.40750353  4.92222288] 0.336810863270943
25000 [-5.36064759 -1.04116353 -0.4112796   4.92972341] 0.33658649107253624
30000 [-5.36367316 -1.04129916 -0.4123533   4.93185743] 0.3365227523633775
35000 [-5.36453492 -1.04133781 -0.41265909  4.93246531] 0.3365046043628453
40000 [-5.36478047 -1.04134883 -0.41274622  4.93263852] 0.33649943381850866
45000 [-5.36485044 -1.04135197 -0.41277105  4.93268788] 0.3364979604069898
50000 [-5.36487038 -1.04135286 -0.41277812  4.93270194] 0.3364975405177562


array([-5.36487038, -1.04135286, -0.41277812,  4.93270194])

In [303]:
def eval_model_l2(X, y, iterations, alpha, lambda_):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + lambda_ * W) 
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [306]:
eval_model_l2(X_st, y, iterations=10000, alpha=1e-1, lambda_ = 1e-2)

1000 [-3.03295741 -0.88655342  0.26084265  3.24693415] 0.39952414661973584
2000 [-3.48715312 -0.89335864  0.08130586  3.53242119] 0.3853552190284769
3000 [-3.56102448 -0.89484106  0.05265943  3.57970098] 0.3831584557334913
4000 [-3.57326894 -0.8950969   0.04792602  3.58756078] 0.3827971622248771
5000 [-3.57530497 -0.89513972  0.04713935  3.58886835] 0.38273716325292756
6000 [-3.5756437  -0.89514685  0.04700848  3.58908591] 0.38272718335942857
7000 [-3.57570007 -0.89514804  0.0469867   3.58912211] 0.3827255229156495
8000 [-3.57570944 -0.89514823  0.04698308  3.58912813] 0.382725246640532
9000 [-3.575711   -0.89514827  0.04698248  3.58912914] 0.3827252006717937
10000 [-3.57571126 -0.89514827  0.04698238  3.5891293 ] 0.38272519302315866


array([-3.57571126, -0.89514827,  0.04698238,  3.5891293 ])